# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [12]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values='NA', schema_overrides={'precip': pl.Float64, 'visib': pl.Float64})

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-745455298.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [19]:
# Write your SQL query here
result = ctx.execute("""
SELECT DISTINCT carrier
FROM airlines
""")

print(result)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [28]:
# Write your SQL query here
result = ctx.execute("""
SELECT dest, COUNT(*) AS flight_count
FROM flights
GROUP BY dest
ORDER BY flight_count DESC
LIMIT 10
""")

print(result)

shape: (10, 2)
┌──────┬──────────────┐
│ dest ┆ flight_count │
│ ---  ┆ ---          │
│ str  ┆ u32          │
╞══════╪══════════════╡
│ ORD  ┆ 17283        │
│ ATL  ┆ 17215        │
│ LAX  ┆ 16174        │
│ BOS  ┆ 15508        │
│ MCO  ┆ 14082        │
│ CLT  ┆ 14064        │
│ SFO  ┆ 13331        │
│ FLL  ┆ 12055        │
│ MIA  ┆ 11728        │
│ DCA  ┆ 9705         │
└──────┴──────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [32]:
# Write your SQL query here
result = ctx.execute("""
SELECT *
FROM flights
WHERE dep_delay > 120
""")

print(result)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [35]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    origin,
    AVG(dep_delay) as avg_flight_delay
FROM flights
WHERE dep_delay IS NOT NULL
GROUP BY origin
ORDER BY avg_flight_delay DESC
""")

print(result)

shape: (3, 2)
┌────────┬──────────────────┐
│ origin ┆ avg_flight_delay │
│ ---    ┆ ---              │
│ str    ┆ f64              │
╞════════╪══════════════════╡
│ EWR    ┆ 15.107954        │
│ JFK    ┆ 12.112159        │
│ LGA    ┆ 10.346876        │
└────────┴──────────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [38]:
# First, let's check what columns are available
result = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")
# print(result)

# Now write your query to find busiest month
result = ctx.execute("""
SELECT month, COUNT(*) AS flight_count
FROM flights
GROUP BY month
ORDER BY flight_count DESC
LIMIT 1
""")

print(result)

shape: (1, 2)
┌───────┬──────────────┐
│ month ┆ flight_count │
│ ---   ┆ ---          │
│ i64   ┆ u32          │
╞═══════╪══════════════╡
│ 7     ┆ 29425        │
└───────┴──────────────┘


### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [45]:
# Write your SQL query here
result = ctx.execute("""
    SELECT carrier,
           CAST(SUM(CASE WHEN dep_delay <= 15 THEN 1 ELSE 0 END) AS REAL) * 100 / COUNT(*) AS pct_on_time
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY carrier
    ORDER BY pct_on_time DESC
""")

print(result)

shape: (16, 2)
┌─────────┬─────────────┐
│ carrier ┆ pct_on_time │
│ ---     ┆ ---         │
│ str     ┆ f64         │
╞═════════╪═════════════╡
│ HA      ┆ 92.982456   │
│ US      ┆ 87.822674   │
│ AS      ┆ 86.797753   │
│ AA      ┆ 84.071293   │
│ DL      ┆ 83.681246   │
│ …       ┆ …           │
│ FL      ┆ 73.32915    │
│ WN      ┆ 73.102706   │
│ F9      ┆ 71.847507   │
│ YV      ┆ 71.376147   │
│ EV      ┆ 69.538126   │
└─────────┴─────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [52]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    f.carrier,
    a.name AS airline_name,
    f.flight,
    f.origin,
    f.dest
FROM flights AS f
JOIN airlines AS a
    ON f.carrier = a.carrier
LIMIT 20
""")

print(result)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [50]:
# Write your SQL query here
result = ctx.execute("""
SELECT flights.carrier,
        AVG(2013 - planes.year) AS avg_plane_age
FROM flights
LEFT JOIN planes
  ON flights.tailnum = planes.tailnum
LEFT JOIN airlines
  ON flights.carrier = airlines.carrier
WHERE planes.year IS NOT NULL
GROUP BY flights.carrier, airlines.name
ORDER BY avg_plane_age DESC
""")

print(result)

shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ MQ      ┆ 35.319        │
│ AA      ┆ 25.869426     │
│ DL      ┆ 16.372169     │
│ UA      ┆ 13.207691     │
│ FL      ┆ 11.385829     │
│ …       ┆ …             │
│ B6      ┆ 6.686702      │
│ F9      ┆ 4.87874       │
│ VX      ┆ 4.473643      │
│ AS      ┆ 3.33662       │
│ HA      ┆ 1.548387      │
└─────────┴───────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [54]:
# First, explore the weather table structure
result = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
# print(result)

# Now write your join query
result = ctx.execute("""
SELECT flights.*
FROM flights
INNER JOIN weather
  ON flights.origin   = weather.origin
  AND flights.time_hour = weather.time_hour
WHERE flights.dep_delay > 0
  AND (weather.precip > 0 OR weather.visib < 5)
""")

print(result)

shape: (16_238, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 6   ┆ 629      ┆ … ┆ 1576     ┆ 5    ┆ 45     ┆ 2013-01-06 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 11  ┆ 11       ┆ … ┆ 1617     ┆ 23   ┆ 59     ┆ 2013-01-12 04:00:00 UTC │
│ 2013 ┆ 1     ┆ 11  ┆ 19       ┆ … ┆ 1576     ┆ 23   ┆ 59     ┆ 2013-01-12 04:00:00 UTC │
│ 2013 ┆ 1     ┆ 11  ┆ 1511     ┆ … ┆ 544      ┆ 15   ┆ 10     ┆ 2013-01-11 20:00:00 UTC │
│ 2013 ┆ 1     ┆ 11  ┆ 1512     ┆ … ┆ 1620     ┆ 15   ┆ 5      ┆ 2013-01-11 20:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …    

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [57]:
# Write your SQL query here
result = ctx.execute("""
SELECT
    p.manufacturer,
    p.model,
    COUNT(*) AS flight_count
FROM flights AS f
JOIN planes AS p
    ON f.tailnum = p.tailnum
WHERE p.manufacturer IS NOT NULL AND p.model IS NOT NULL
GROUP BY p.manufacturer, p.model
ORDER BY flight_count DESC
LIMIT 10
""")

print(result)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬──────────────┐
│ manufacturer                  ┆ model           ┆ flight_count │
│ ---                           ┆ ---             ┆ ---          │
│ str                           ┆ str             ┆ u32          │
╞═══════════════════════════════╪═════════════════╪══════════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278        │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027        │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716        │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553        │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051        │
│ BOEING                        ┆ 737-824         ┆ 13809        │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807        │
│ BOEING                        ┆ 737-7H4         ┆ 10389        │
│ BOEING                        ┆ 757-222         ┆ 9150         │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88        

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [58]:
# Write your SQL query here
result = ctx.execute("""
WITH enriched AS (
    SELECT flights.*,
            ao.name AS origin_airport,
            ad.name AS destination_airport
    FROM flights
    LEFT JOIN airports AS ao ON flights.origin = ao.faa
    LEFT JOIN airports AS ad ON flights.dest   = ad.faa
)
SELECT origin_airport,
        destination_airport,
        COUNT(*) AS total_flights,
        AVG(dep_delay) AS average_departure_delay,
        CAST(AVG(CASE WHEN dep_delay > 30 THEN 1 ELSE 0 END) * 100 AS REAL)
          AS percentage_delayed_more_than_30_min
FROM enriched
WHERE dep_delay IS NOT NULL
GROUP BY origin_airport, destination_airport
ORDER BY total_flights DESC
LIMIT 10
""")

print(result)

shape: (10, 5)
┌────────────────┬─────────────────────┬───────────────┬─────────────────────┬─────────────────────┐
│ origin_airport ┆ destination_airport ┆ total_flights ┆ average_departure_d ┆ percentage_delayed_ │
│ ---            ┆ ---                 ┆ ---           ┆ elay                ┆ more_than_3…        │
│ str            ┆ str                 ┆ u32           ┆ ---                 ┆ ---                 │
│                ┆                     ┆               ┆ f64                 ┆ f32                 │
╞════════════════╪═════════════════════╪═══════════════╪═════════════════════╪═════════════════════╡
│ John F Kennedy ┆ Los Angeles Intl    ┆ 11196         ┆ 8.522508            ┆ 9.88746             │
│ Intl           ┆                     ┆               ┆                     ┆                     │
│ La Guardia     ┆ Hartsfield Jackson  ┆ 10082         ┆ 11.448621           ┆ 12.467764           │
│                ┆ Atlanta Int…        ┆               ┆                    

In [ ]:
# Write your SQL query here
result = ctx.execute("""
-- Your query here
""")

# print(result)

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [59]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here

SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘


In [60]:
# Bonus Question (3.3 in Polars)

flights_weather = flights.join(
    weather, on=["origin", "time_hour"], how="inner"
)

delayed_and_bad = flights_weather.filter(
    (pl.col("dep_delay") > 0) & ((pl.col("precip") > 0) | (pl.col("visib") < 5))
)

delayed_and_bad


year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,year_right,month_right,day_right,hour_right,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib
i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,str,str,i64,i64,i64,i64,"datetime[μs, UTC]",i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,f64,f64
2013,1,6,629,545,44,1156,1022,94,"""B6""",725,"""N661JB""","""JFK""","""BQN""",202,1576,5,45,2013-01-06 10:00:00 UTC,2013,1,6,5,33.08,30.92,91.66,250,8.05546,null,0.0,1020.9,4.0
2013,1,11,11,2359,12,436,444,-8,"""B6""",739,"""N779JB""","""JFK""","""PSE""",190,1617,23,59,2013-01-12 04:00:00 UTC,2013,1,11,23,46.04,44.96,95.99,50,8.05546,null,0.02,1019.2,3.0
2013,1,11,19,2359,20,439,437,2,"""B6""",727,"""N705JB""","""JFK""","""BQN""",180,1576,23,59,2013-01-12 04:00:00 UTC,2013,1,11,23,46.04,44.96,95.99,50,8.05546,null,0.02,1019.2,3.0
2013,1,11,1511,1510,1,1730,1710,20,"""MQ""",4579,"""N524MQ""","""LGA""","""CLT""",104,544,15,10,2013-01-11 20:00:00 UTC,2013,1,11,15,39.92,35.06,82.6,50,9.20624,null,0.01,1025.0,9.0
2013,1,11,1512,1505,7,1823,1734,49,"""UA""",745,"""N528UA""","""LGA""","""DEN""",270,1620,15,5,2013-01-11 20:00:00 UTC,2013,1,11,15,39.92,35.06,82.6,50,9.20624,null,0.01,1025.0,9.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2013,9,21,1249,1245,4,1541,1600,-19,"""AA""",1697,"""N618AA""","""JFK""","""MIA""",135,1089,12,45,2013-09-21 16:00:00 UTC,2013,9,21,12,71.6,68.0,88.43,180,16.11092,null,0.01,null,10.0
2013,9,21,1252,1159,53,1612,1453,79,"""B6""",323,"""N516JB""","""JFK""","""LAX""",323,2475,11,59,2013-09-21 15:00:00 UTC,2013,9,21,11,68.0,66.02,93.38,170,10.35702,null,0.17,1012.5,6.0
2013,9,21,1257,1247,10,1555,1535,20,"""B6""",553,"""N597JB""","""JFK""","""PBI""",132,1028,12,47,2013-09-21 16:00:00 UTC,2013,9,21,12,71.6,68.0,88.43,180,16.11092,null,0.01,null,10.0
